In [30]:
import numpy
import time
import pickle
from __future__ import print_function

In [31]:

def load_file(filepath):
    
    with open(filepath, 'r') as f:
        file = f.readlines()
    return file

src_file = load_file('data/file.en')
trg_file = load_file('data/file.de')
aligned_file = load_file('data/file.aligned')

In [32]:
def extract(trg_start, trg_end, src_start, src_end, alignments, trg_aligned, trg_len):    
  

    if trg_end < 0 :
        return [], []
    
    A = []
    
    a = []
    
    for trg_word, src_word  in alignments:
        if trg_start <= trg_word <= trg_end:
            
            if (src_word < src_start or src_word > src_end):  
                return [], []
            else:
                
                a.append( (trg_word - trg_start, src_word - src_start) )

    E = []
    trg_s = trg_start
    
    while True:
        trg_e = trg_end
        while True:
            #E.append( (tuple(trg_words[trg_s:trg_e+1]) , tuple(src_words[src_start:src_end+1])) )
            E.append( (" ".join(trg_words[trg_s:trg_e+1]) , " ".join(src_words[src_start:src_end+1]) ))
            
            A.append(a)
             
            trg_e += 1
            
            if trg_e in trg_aligned or trg_e == len(trg_words):
                break 

        trg_s -= 1
        
        if trg_s in trg_aligned or trg_s < 0:
            break
       
        a = [(i[0]+1,i[1]) for i in a]
        
    return E, A


def extract_phrases(src_words, trg_words, alignments, cutoff):
    
    
    if cutoff == -1:
        cutoff = len(src_words)
        
        
    trg_aligned = set()
    for (trg,_) in alignments:
        trg_aligned.add(trg)
    
    
    extracted_phrases = []
    extracted_alignments = []
    for src_start in range(len(src_words)):
        for src_end in range(src_start, min(src_start + cutoff, len(src_words))):

            trg_start = len(trg_words) - 1
            trg_end = - 1
            for  (trg,src) in alignments: 
                if src_start <= src <= src_end:
                    trg_start = min(trg, trg_start)
                    trg_end = max(trg, trg_end)
                    
            
            if(trg_end - trg_start > cutoff - 1):
                continue

            phrase_pairs, A = extract(trg_start, trg_end, src_start, src_end, alignments, trg_aligned, len(trg_words))
            
            if (phrase_pairs):
                extracted_phrases.extend(phrase_pairs)
                for a in A:
                    extracted_alignments.append(a);
            
    return extracted_phrases, extracted_alignments

In [33]:
def add_freq_dict(dictionary, item):
    if item in dictionary:
        dictionary[item] += 1
    else:
        dictionary[item] = 1

        

freq_pairs = dict()
freq_src = dict()
freq_trg = dict()
    
start_time = time.time()
    
for i in range(len(src_file)):
    src_words = src_file[i].split()
    trg_words = trg_file[i].split()        
    alignments = [[int(a) for a in alignment.split('-')] for alignment in aligned_file[i].split()]
    
    phrase_pairs, extracted_alignments = extract_phrases(src_words, trg_words, alignments, 5)
     
    
    for j, pair in enumerate(phrase_pairs):
        
        a = extracted_alignments[j]
        
        if pair in freq_pairs:
            freq_pairs[pair][0] += 1;
        else:
             freq_pairs[pair] = [1, a]
        
        
        add_freq_dict(freq_trg, pair[0])
        add_freq_dict(freq_src, pair[1])
        
    print(' \r%d / %d (%ds)'%(i+1,len(src_file), time.time() - start_time), end = '')

print()
print('Done!!!')
print('Total duration: %ds'%(time.time() - start_time))

print("#en phrases: %d"%(len(freq_src)))
print("#de phrases: %d"%(len(freq_trg)))
print("#(en,de) phrases: %d"%(len(freq_pairs)))

50000 / 50000 (73s)                                                                                                                                                                                                                                                                                                                                               
Done!!!
Total duration: 73s
#en phrases: 1189230
#de phrases: 1386442
#(en,de) phrases: 2624229


In [5]:
for key in freq_pairs.keys()[10:20]:
    print(key, freq_pairs[key])

('darauf , wie die tiere', 'to the transfer of animals') [1, [(0, 0), (1, 0), (3, 1), (2, 2), (4, 4)]]
('geht', 'is needed') [1, [(0, 0)]]
('es ist nicht richtig , wenn hier', 'you are wrong to') [1, [(0, 0), (1, 1), (2, 2), (3, 2), (4, 3)]]
('auf die kommission an', "in the commission 's court") [1, [(0, 0), (1, 1), (2, 2), (2, 3), (3, 3), (3, 4)]]
('der ausgangspunkt', 'the point') [1, [(0, 0), (1, 1)]]
('die erarbeitung', 'communication') [1, [(0, 0)]]
('widersinnig und nicht', 'nonsensical and contrary to') [1, [(0, 0), (1, 1), (0, 2), (2, 2)]]
('informieren und dort eine kritik anzubringen', 'and then levy criticism') [1, [(1, 0), (2, 1), (3, 2), (5, 2), (4, 3)]]
('entwicklungspolitik steht im zusammenhang', 'development policies relates') [1, [(0, 0), (0, 1), (1, 2), (2, 2), (3, 2)]]
('f\xc3\xbcr die umsetzungsmodalit\xc3\xa4ten der', 'for the implementation methods for') [1, [(0, 0), (1, 1), (2, 2), (2, 3), (2, 4), (3, 4)]]


### Load/save dictionaries

In [ ]:
#Save files
# with open("data/freq_src_small", 'wb') as file:
#     pickle.dump(freq_src, file)

# with open("data/freq_trg_small", 'wb') as file:
#     pickle.dump(freq_trg, file)

# with open("data/freq_pairs_small", 'wb') as file:
#     pickle.dump(freq_pairs, file)

with open("data/joined_freq_src", 'rb') as file:
    freq_src = pickle.load(file)

with open("data/joined_freq_trg", 'rb') as file:
    freq_trg = pickle.load(file)

with open("data/joined_freq_pairs", 'rb') as file:
    freq_pairs = pickle.load(file)
        
print("#en phrases: %d"%(len(freq_src)))
print("#de phrases: %d"%(len(freq_trg)))
print("#(en,de) phrases: %d"%(len(freq_pairs)))     

In [ ]:
# import operator
# sorted_freq_src = sorted(freq_src.items(), key=operator.itemgetter(1), reverse = True)
# sorted_freq_trg = sorted(freq_trg.items(), key=operator.itemgetter(1), reverse = True)
# sorted_freq_pairs = sorted(freq_pairs.items(), key=operator.itemgetter(1), reverse = True)

## Calculate p(e|f) and p(f|e)

In [34]:
src_total = 0
for _, freq in freq_src.iteritems():
    src_total += freq
    
trg_total = 0
for _, freq in freq_trg.iteritems():
    trg_total += freq
    
pairs_total = 0
for _, freq in freq_pairs.iteritems():
    pairs_total += freq[0]

print("#en phrases: %d"%(src_total))
print("#de phrases: %d"%(trg_total))
print("#(en,de) phrases: %d"%(pairs_total))     

#en phrases: 3947866
#de phrases: 3947866
#(en,de) phrases: 3947866


In [35]:
for trg_src_pair, value in freq_pairs.iteritems():
    freq = value[0]
    
    p_src_trg = 1.0 * freq / freq_trg[trg_src_pair[0]]
    p_trg_src = 1.0 * freq / freq_src[trg_src_pair[1]]
    
    freq_pairs[trg_src_pair] = [freq, value[1], p_src_trg, p_trg_src]
    

### KMO

In [45]:
c = 1
start_time = time.time()

for trg_src_pair, value in freq_pairs.iteritems():
    pair_freq = value[0]
    a = value[1]
    trg_words = trg_src_pair[0].split()
    src_words = trg_src_pair[1].split()
    
    mat = len(trg_words) * [None]
    
    for (trg_pos, src_pos) in a:
        
        src_word = src_words[src_pos]
        trg_word = trg_words[trg_pos]
        
        if mat[trg_pos] is None:
            mat[trg_pos] = []
        
        if (trg_word, src_word) in freq_pairs:
            w = freq_pairs[(trg_word, src_word)][3]
            
            mat[trg_pos].append(w)
            
        else:
            mat[trg_pos].append(0)
            
    for i in range(len(mat)):
        if mat[i] is None:
            mat[i] = 1
        else:
            mat[i] = numpy.mean(mat[i])
    
    lex = numpy.sum(numpy.log(mat))
    
    freq_pairs[trg_src_pair].append(lex)
        
    
    print(' \r%d / %d (%ds)'%(c,len(freq_pairs), time.time() - start_time), end = '')
    c +=1
        
        
    
    

861 / 2624229 (0s)

C:\Users\Thanos\Anaconda2\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in log


1858 / 2624229 (0s)

3056 / 2624229 (0s)

4533 / 2624229 (0s)

...

69732 / 2624229 (15s)

71260 / 2624229 (15s)

72739 / 2624229 (16s)

73819 / 2624229 (16s)

KeyboardInterrupt: 

In [44]:
for key in freq_pairs.keys()[10:20]:
    print(key, freq_pairs[key])

('darauf , wie die tiere', 'to the transfer of animals') [1, [(0, 0), (1, 0), (3, 1), (2, 2), (4, 4)], 1.0, 1.0, 6.4882145318840187e-07, -14.248108268741809]
('geht', 'is needed') [1, [(0, 0)], 0.001451378809869376, 0.01694915254237288, 0.01154058531198233, -4.4618852989126481]
('es ist nicht richtig , wenn hier', 'you are wrong to') [1, [(0, 0), (1, 1), (2, 2), (3, 2), (4, 3)], 1.0, 0.3333333333333333, 1.0704637144922731e-08, -18.352588811329525]
('auf die kommission an', "in the commission 's court") [1, [(0, 0), (1, 1), (2, 2), (2, 3), (3, 3), (3, 4)], 1.0, 0.5, 9.9148749667829182e-07, -13.824059499577126]
('der ausgangspunkt', 'the point') [1, [(0, 0), (1, 1)], 0.2, 0.009708737864077669, 0.0003497275527715728, -7.9583561272601377]
('die erarbeitung', 'communication') [1, [(0, 0)], 0.021739130434782608, 0.0037593984962406013, 0.0037593984962406013, -5.5834963087816991]
('widersinnig und nicht', 'nonsensical and contrary to') [1, [(0, 0), (1, 1), (0, 2), (2, 2)], 0.5, 1.0, 0.0, -inf]

In [48]:
freq_pairs[('nicht', 'not a')]

[22, [(0, 0)], 0.0018144329896907217, 0.11055276381909548, 0.64859525899912207]